In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, concat, read_csv
sns.set_theme(style='white', context='notebook', font_scale=1.33)

## Section 1: Factor Loadings

### 1.1 One-factor model

In [2]:
from pandas import DataFrame

## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
summary = []
for study in studies:
    df = read_csv(os.path.join('stan_results', study, 'grmq_m1_summary.tsv'), sep='\t', index_col=0)    
    df = df.T.filter(regex='lambda\[[0-9]*,1\]').T['Mean']
    summary.append(df)
    
## Convert to DataFrame.
summary = DataFrame(summary).T
summary.columns = studies

## Print summary.
summary.describe().round(3)

,teicher2015,tuominen2022
count,40.000,40.000
mean,0.601,0.602
std,0.122,0.109
min,0.270,0.350
25%,0.528,0.528
50%,0.595,0.605
75%,0.680,0.673
max,0.860,0.830


### 1.2 Bifactor Model (10 groups)

In [3]:
from pandas import DataFrame

## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
summary = []
for study in studies:
    df = read_csv(os.path.join('stan_results', study, 'grmq_m2_summary.tsv'), sep='\t', index_col=0)    
    df = df.T.filter(regex='lambda\[[0-9]*,1\]').T['Mean']
    summary.append(df)
    
## Convert to DataFrame.
summary = DataFrame(summary).T
summary.columns = studies

## Print summary.
summary.describe().round(3)

,teicher2015,tuominen2022
count,40.000,40.000
mean,0.582,0.586
std,0.146,0.125
min,0.250,0.290
25%,0.467,0.500
50%,0.605,0.595
75%,0.692,0.673
max,0.870,0.830


In [4]:
from pandas import DataFrame

## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
summary = []
for study in studies:
    df = read_csv(os.path.join('stan_results', study, 'grmq_m2_summary.tsv'), sep='\t', index_col=0)    
    df = df.T.filter(regex='lambda\[[0-9]*,[^1]|lambda\[[0-9]*,1[0-2]').T['Mean']
    summary.append(df)
    
## Convert to DataFrame.
summary = DataFrame(summary).T
summary.columns = studies

## Print summary.
summary.describe().round(3)

,teicher2015,tuominen2022
count,40.000,40.000
mean,0.320,0.350
std,0.202,0.187
min,0.015,0.044
25%,0.165,0.178
50%,0.285,0.340
75%,0.445,0.490
max,0.800,0.700


### 1.3 Bifactor S-1 Model (3 groups)

In [5]:
from pandas import DataFrame

## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
summary = []
for study in studies:
    df = read_csv(os.path.join('stan_results', study, 'grmq_m3_summary.tsv'), sep='\t', index_col=0)    
    df = df.T.filter(regex='lambda\[[0-9]*,1\]').T['Mean']
    summary.append(df)
    
## Convert to DataFrame.
summary = DataFrame(summary).T
summary.columns = studies

## Print summary.
summary.describe().round(3)

,teicher2015,tuominen2022
count,40.000,40.000
mean,0.569,0.577
std,0.169,0.132
min,0.160,0.290
25%,0.430,0.470
50%,0.595,0.580
75%,0.692,0.670
max,0.870,0.860


In [7]:
from pandas import DataFrame

## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
summary = []
for study in studies:
    df = read_csv(os.path.join('stan_results', study, 'grmq_m3_summary.tsv'), sep='\t', index_col=0)    
    df = df.T.filter(regex='lambda\[[0-9]*,[^1]|lambda\[[0-9]*,1[0-2]').T['Mean']
    summary.append(df)
    
## Convert to DataFrame.
summary = DataFrame(summary).T
summary.columns = studies

## Print summary.
summary.describe().round(3)

,teicher2015,tuominen2022
count,15.000,15.000
mean,0.551,0.527
std,0.124,0.145
min,0.330,0.190
25%,0.480,0.405
50%,0.570,0.560
75%,0.590,0.635
max,0.790,0.690


### 1.4 Table 2

In [ ]:
from pandas import Categorical

## Load design data.
design = read_csv(os.path.join('data', 'design.csv'), index_col=0)
design = design[design.columns[:11]]

## Define locally dependent items.
ld = [[7,8], [9,10,11], [13,14], [15,16], [19,20], [21,22,23], [24,25], [33,34,35], [36,37]]
for ix in ld: design = design.drop(index=ix[1:])
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load Stan summary.
m1 = read_csv(os.path.join('stan_results', 'teicher2015', 'grmq_m1_summary.tsv'), sep='\t', index_col=0)
m2 = read_csv(os.path.join('stan_results', 'teicher2015', 'grmq_m2_summary.tsv'), sep='\t', index_col=0)

## Extract factor loadings.
loadings = np.zeros((len(design), 2)).astype(float)
for i, j in np.column_stack([np.where(design)]).T:
    loadings[i,int(j > 0)] = m2.loc[f'lambda[{i+1},{j+1}]','Mean']
     
## Merge with unidimensional model.
loadings = np.column_stack([
    m1.T.filter(regex='lambda').T['Mean'].values,
    loadings
])
        
## Convert to DataFrame.
design = design.drop(columns='general')
loadings = DataFrame(loadings, columns=['uni','general', 'group'])
loadings.insert(0, 'item', design.index)
loadings.insert(0, 'subscale', design.columns[np.where(design.values)[-1]])

## Sort DataFrame.
cols = ['PVA','PPhysA','NVEA','SexA','EN','PN','WSV','WIPV','PeerVA','PeerPhysA']
loadings['subscale'] = Categorical(loadings.subscale, categories=cols, ordered=True)
loadings = loadings.sort_values(['subscale','item'])

## Format columns.
loadings['item'] = loadings.item.apply(lambda x: '%0.0f' %x)
loadings['uni'] = loadings.uni.apply(lambda x: '%0.3f' %x)
loadings['general'] = loadings.general.apply(lambda x: '%0.3f' %x)
loadings['group'] = loadings.group.apply(lambda x: '%0.3f' %x)

## Convert to pivot table.
aggfunc = lambda x: ''.join(x)
loadings.insert(0, 'col', np.arange(len(loadings)) // 20)
loadings.insert(0, 'row', np.arange(len(loadings)) % 20)
loadings = loadings.pivot_table(['subscale','item','uni','general','group'], 'row', 'col', aggfunc=aggfunc).fillna('')
loadings['subscale'] = loadings['subscale'].apply(lambda x: np.where(x == np.roll(x,1), '', x))

## Sort columns.
loadings = loadings[['subscale', 'item','uni','general','group']]
loadings = loadings.swaplevel(0, 1, 1)
loadings = loadings[[0,1]]

## Display table.
loadings

In [ ]:
print(loadings.to_latex(index=False))

## Section 3: Variance Decomposition

In [ ]:
## Load design data.
design = read_csv(os.path.join('data', 'design.csv'), index_col=0)

## Define locally dependent items.
ld = [[7,8], [9,10,11], [13,14], [15,16], [19,20], [21,22,23], [24,25], [33,34,35], [36,37]]
for ix in ld: design = design.drop(index=ix[1:])

### 3.1 Bifactor model

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define I/O parameters.
studies = ['teicher2015', 'tuominen2022']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Restrict to columns of interest.
D = design[design.columns[:11]].copy()

stats = []
for study in studies:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load Stan summary.
    summary = read_csv(os.path.join('stan_results', study, 'grmq_m2_summary.tsv'), sep='\t', index_col=0)
    
    ## Extract factor loadings.
    loadings = np.zeros_like(D).astype(float)
    for i, j in np.column_stack([np.where(D)]).T:
        loadings[i,j] = summary.loc[f'lambda[{i+1},{j+1}]','Mean']
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Coefficient omega hierachical.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Preallocate space.
    omega   = np.zeros(len(D.columns))
    omega_s = np.zeros(len(D.columns))
        
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Restrict to items in group.
        L = loadings[D[col]==1]
        
        ## Compute squared sum of factor loadings.
        A = np.square(np.sum(L, axis=0))
        
        ## Compute sum of error variances.
        B = np.sum(1 - np.square(L).sum(axis=1))
        
        ## Compute total variance.
        C = np.sum(A) + B
        
        ## Compute coefficient omega.
        omega[i] = A.sum() / C
        
        ## Compute coefficient omega subscale.
        omega_s[i] = A[i] / C
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Explained common variance.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Compute sum of squares.
    ss = np.square(loadings).sum(axis=0)
    
    ## Compute explained common variance.
    ecv = ss / ss.sum()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### H-index
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Preallocate space.
    H = np.zeros(len(D.columns))
    
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Compute squared loadings.
        s = np.square(loadings[:,i])
        
        ## Compute H-index.
        H[i] = 1. / (1 + 1 / np.sum(s / (1-s)))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Convert to DataFrame.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats.append(DataFrame(dict(
        subscale = D.columns,
        study = np.repeat(study, D.columns.size),
        ecv = ecv,
        omega = omega,
        omega_s = omega_s,
        H = H
    )))
    
## Concatenate DataFrames.
stats = concat(stats).replace({'teicher2015':1, 'tuominen2022': 2})

## Convert to pivot table.
stats = stats.pivot_table(['omega','omega_s','ecv','H'], 'subscale', 'study').round(3)
stats = stats.astype(str)
stats.loc[stats.index!='general','ecv'] = ''

## Re-organize rows.
index = ['general', 'PVA', 'PPhysA', 'NVEA', 'SexA', 'EN', 'PN', 'WSV', 'WIPV', 'PeerVA', 'PeerPhysA']
stats = stats.loc[index]

## Re-organize columns.
cols = [(1,'ecv'),(1,'omega'),(1,'omega_s'),(1,'H'),(2,'ecv'),(2,'omega'),(2,'omega_s'),(2,'H')]
stats = stats.swaplevel(axis='columns')[cols]

## Display table.
stats

In [ ]:
print(stats.to_latex())

### 3.2 Bifactor S-1 model

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define I/O parameters.
studies = ['teicher2015', 'tuominen2022']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Restrict to columns of interest.
D = design[['general','peer','reverse']].copy()

stats = []
for study in studies:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load Stan summary.
    summary = read_csv(os.path.join('stan_results', study, 'grmq_m3_summary.tsv'), sep='\t', index_col=0)
    
    ## Extract factor loadings.
    loadings = np.zeros_like(D).astype(float)
    for i, j in np.column_stack([np.where(D)]).T:
        loadings[i,j] = summary.loc[f'lambda[{i+1},{j+1}]','Mean']
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Coefficient omega hierachical.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Preallocate space.
    omega   = np.zeros(len(D.columns))
    omega_s = np.zeros(len(D.columns))
        
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Restrict to items in group.
        L = loadings[D[col]==1]
        
        ## Compute squared sum of factor loadings.
        A = np.square(np.sum(L, axis=0))
        
        ## Compute sum of error variances.
        B = np.sum(1 - np.square(L).sum(axis=1))
        
        ## Compute total variance.
        C = np.sum(A) + B
        
        ## Compute coefficient omega.
        omega[i] = A.sum() / C
        
        ## Compute coefficient omega subscale.
        omega_s[i] = A[i] / C
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Explained common variance.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Compute sum of squares.
    ss = np.square(loadings).sum(axis=0)
    
    ## Compute explained common variance.
    ecv = ss / ss.sum()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### H-index
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Preallocate space.
    H = np.zeros(len(D.columns))
    
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Compute squared loadings.
        s = np.square(loadings[:,i])
        
        ## Compute H-index.
        H[i] = 1. / (1 + 1 / np.sum(s / (1-s)))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Convert to DataFrame.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats.append(DataFrame(dict(
        subscale = D.columns,
        study = np.repeat(study, D.columns.size),
        ecv = ecv,
        omega = omega,
        omega_s = omega_s,
        H = H
    )))
    
## Concatenate DataFrames.
stats = concat(stats).replace({'teicher2015':1, 'tuominen2022': 2})

## Convert to pivot table.
stats = stats.pivot_table(['omega','omega_s','ecv','H'], 'subscale', 'study').round(3)
stats = stats.astype(str)
stats.loc[stats.index!='general','ecv'] = ''

## Re-organize rows.
index = D.columns
stats = stats.loc[index]

## Re-organize columns.
cols = [(1,'ecv'),(1,'omega'),(1,'omega_s'),(1,'H'),(2,'ecv'),(2,'omega'),(2,'omega_s'),(2,'H')]
stats = stats.swaplevel(axis='columns')[cols]

## Display table.
stats

In [ ]:
print(stats.to_latex())